In [1]:
import os
import zipfile
import pydicom
import numpy as np
import pyvista as pv
from scipy.ndimage import gaussian_filter
from skimage.restoration import denoise_bilateral  # IMPORTAR la función denoise_bilateral
import pydicom
import matplotlib.pyplot as plt
from scipy.ndimage import sobel
import cv2
from scipy.interpolate import make_interp_spline
from scipy.optimize import curve_fit
%matplotlib inline
from skimage.filters import threshold_otsu
from sklearn.linear_model import RANSACRegressor
from skimage.filters import threshold_otsu
from sklearn.linear_model import RANSACRegressor
from ipywidgets import interact
import json
from ultralytics import YOLO

In [2]:
# Ruta al archivo ZIP
zip_file = 'C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/dicom.zip'
extract_folder = 'C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/descomprimido/'
output_folder = 'C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/jpeg_images/'  # Carpeta de salida

# Crear carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Verificar si la carpeta ya está descomprimida
if not os.path.exists(extract_folder) or not any(f.endswith('.dcm') for f in os.listdir(extract_folder)):
    print("Descomprimiendo archivos...")
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
    print("Descompresión completada.")
else:
    print("La carpeta ya está descomprimida")

# Listar archivos DICOM
dicom_files = sorted([os.path.join(extract_folder, f) for f in os.listdir(extract_folder) if f.endswith('.dcm')])

if not dicom_files:
    print("No se encontraron archivos DICOM en la carpeta.")
    exit()

# Convertir cada imagen DICOM a JPEG
for dicom_file in dicom_files:
    ds = pydicom.dcmread(dicom_file)
    img_array = ds.pixel_array  # Obtener la imagen en escala de grises

    # Normalizar la imagen a rango [0, 255] para guardar como JPEG
    img_array = (img_array - np.min(img_array)) / (np.max(img_array) - np.min(img_array)) * 255
    img_array = img_array.astype(np.uint8)

    # Obtener el nombre original sin extensión
    file_name = os.path.basename(dicom_file).replace('.dcm', '.jpeg')

    # Ruta de salida
    output_path = os.path.join(output_folder, file_name)

    # Guardar la imagen en formato JPEG
    cv2.imwrite(output_path, img_array)

print(f"✅ {len(dicom_files)} imágenes convertidas y guardadas en {output_folder}")

La carpeta ya está descomprimida
✅ 311 imágenes convertidas y guardadas en C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/jpeg_images/


In [3]:
import os
import cv2
import json

# Rutas de carpetas
image_folder = "C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/jpeg_images/"
output_folder = "C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/annotated_images/"
annotations_file = "C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/annotations.json"

# Crear carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Lista de imágenes
image_files = sorted([f for f in os.listdir(image_folder) if f.endswith(".jpeg")])

# Variables globales
boxes = []
drawing = False
x_init, y_init = -1, -1
current_color = (0, 0, 255)  # Rojo por defecto

# Cargar anotaciones previas si existen
if os.path.exists(annotations_file):
    with open(annotations_file, "r") as f:
        annotations = json.load(f)
else:
    annotations = {}

# Función para dibujar los rectángulos
def draw_rectangle(event, x, y, flags, param):
    global x_init, y_init, drawing, img, boxes, current_color

    if event == cv2.EVENT_LBUTTONDOWN:  # Inicio del dibujo
        drawing = True
        x_init, y_init = x, y

    elif event == cv2.EVENT_LBUTTONUP:  # Fin del dibujo
        drawing = False
        cv2.rectangle(img, (x_init, y_init), (x, y), current_color, 2)
        label = "superior" if current_color == (0, 0, 255) else "inferior"
        boxes.append((x_init, y_init, x, y, label))

# Función para procesar una imagen
def annotate_image(image_index):
    global img, boxes, current_color

    img_path = os.path.join(image_folder, image_files[image_index])
    img = cv2.imread(img_path)
    boxes = []

    print(f"📌 Anotando imagen: {image_files[image_index]}")
    print("🔴 Presiona 'r' para marcar arco superior (rojo)")
    print("🔵 Presiona 'b' para marcar arco inferior (azul)")
    print("💾 Presiona 's' para guardar y pasar a la siguiente imagen")
    print("❌ Presiona 'q' para salir sin guardar")

    cv2.namedWindow("Anotador de Arcos Dentales")
    cv2.setMouseCallback("Anotador de Arcos Dentales", draw_rectangle)

    while True:
        temp_img = img.copy()
        for (x1, y1, x2, y2, label) in boxes:
            color = (0, 0, 255) if label == "superior" else (255, 0, 0)
            cv2.rectangle(temp_img, (x1, y1), (x2, y2), color, 2)

        cv2.imshow("Anotador de Arcos Dentales", temp_img)
        key = cv2.waitKey(1) & 0xFF

        if key == ord("r"):
            current_color = (0, 0, 255)
            print("🔴 Seleccionado: Arco Superior")
        elif key == ord("b"):
            current_color = (255, 0, 0)
            print("🔵 Seleccionado: Arco Inferior")

        elif key == ord("s"):
            # Dibujar las cajas sobre la imagen antes de guardar
            for (x1, y1, x2, y2, label) in boxes:
                color = (0, 0, 255) if label == "superior" else (255, 0, 0)
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

            output_path = os.path.join(output_folder, image_files[image_index])
            cv2.imwrite(output_path, img)

            # Guardar las coordenadas en JSON
            annotations[image_files[image_index]] = boxes
            with open(annotations_file, "w") as f:
                json.dump(annotations, f, indent=4)

            print(f"✅ Imagen guardada en: {output_path}")
            print(f"📝 Anotaciones guardadas en {annotations_file}")
            break

        elif key == ord("q"):
            break

    cv2.destroyAllWindows()

# Ejecutar anotación en todas las imágenes
for index in range(len(image_files)):
    annotate_image(index)


📌 Anotando imagen: image0001.jpeg
🔴 Presiona 'r' para marcar arco superior (rojo)
🔵 Presiona 'b' para marcar arco inferior (azul)
💾 Presiona 's' para guardar y pasar a la siguiente imagen
❌ Presiona 'q' para salir sin guardar
✅ Imagen guardada en: C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/annotated_images/image0001.jpeg
📝 Anotaciones guardadas en C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/annotations.json
📌 Anotando imagen: image0002.jpeg
🔴 Presiona 'r' para marcar arco superior (rojo)
🔵 Presiona 'b' para marcar arco inferior (azul)
💾 Presiona 's' para guardar y pasar a la siguiente imagen
❌ Presiona 'q' para salir sin guardar
✅ Imagen guardada en: C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/annotated_images/image0002.jpeg
📝 Anotaciones guardadas en C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/annotations.json
📌 Anotando imagen: image0003.jpeg
🔴 Presiona 'r' para marcar arco superior (rojo)
🔵 Presiona 'b' para marcar arco inferior (azul)
💾 Presiona

In [4]:
# Directorios
yolo_labels_folder = "C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/yolo_labels/"

# Crear carpeta para las etiquetas YOLO si no existe
os.makedirs(yolo_labels_folder, exist_ok=True)

# Cargar anotaciones
with open(annotations_file, "r") as f:
    annotations = json.load(f)

# Obtener tamaño de las imágenes (necesario para normalizar las coordenadas)
def get_image_size(image_path):
    import cv2
    img = cv2.imread(image_path)
    h, w, _ = img.shape
    return w, h

# Procesar cada imagen anotada
for image_name, boxes in annotations.items():
    image_path = os.path.join(image_folder, image_name)
    
    if not os.path.exists(image_path):
        print(f"⚠️ Imagen no encontrada: {image_name}, omitiendo...")
        continue
    
    # Obtener tamaño de la imagen
    img_w, img_h = get_image_size(image_path)
    
    # Crear el archivo YOLO correspondiente
    yolo_file_path = os.path.join(yolo_labels_folder, image_name.replace(".jpeg", ".txt"))
    with open(yolo_file_path, "w") as yolo_file:
        for (x1, y1, x2, y2, label) in boxes:
            class_id = 0 if label == "superior" else 1  # 0 = arco superior, 1 = arco inferior
            
            # Convertir a formato YOLO
            x_center = (x1 + x2) / 2 / img_w
            y_center = (y1 + y2) / 2 / img_h
            width = abs(x2 - x1) / img_w
            height = abs(y2 - y1) / img_h
            
            # Escribir en el archivo
            yolo_file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    print(f"✅ Etiqueta YOLO generada: {yolo_file_path}")

print("🚀 Conversión a YOLO completada.")


✅ Etiqueta YOLO generada: C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/yolo_labels/image0001.txt
✅ Etiqueta YOLO generada: C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/yolo_labels/image0002.txt
✅ Etiqueta YOLO generada: C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/yolo_labels/image0003.txt
✅ Etiqueta YOLO generada: C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/yolo_labels/image0004.txt
✅ Etiqueta YOLO generada: C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/yolo_labels/image0005.txt
✅ Etiqueta YOLO generada: C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/yolo_labels/image0006.txt
✅ Etiqueta YOLO generada: C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/yolo_labels/image0007.txt
✅ Etiqueta YOLO generada: C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/yolo_labels/image0008.txt
✅ Etiqueta YOLO generada: C:/Users/ana.torres/Desktop/python/biomod/DICOM/casX/yolo_labels/image0009.txt
✅ Etiqueta YOLO generada: C:/Users/ana.torres/Desktop/p